In [ ]:
import pandas as pd

rel_mowa_df = pd.read_csv('/../data_mowa.csv', sep=';') # transcribed statements

rel_pismo_df = pd.read_csv('/../data_pismo.csv', sep=';') # typed statements

In [ ]:
import codecs

all_texts = {} ; all_labels = {}

fpath = '.../typed_statements/'

all_labels['p'] = [] ; all_texts['p'] = []
for index, row in rel_pismo_df.iterrows():
  if row['id']+'.txt' in pisemne_files:
    if row['credibility'] >= 2: all_labels['p'].append(1)
    else: all_labels['p'].append(0)
     
    with codecs.open(fpath+row['id']+'.txt','r','utf8') as fr:
      all_texts['p'].append ( fr.read() )

print(len(all_texts['p']))


mowa_fname_creds = {} ; mowa_fname_texts = {}

fpath = '.../transcribed_statements/'

all_labels['t'] = [] ; all_texts['t'] = []
for index, row in rel_mowa_df.iterrows():
  if row['id']+'.txt' in transkrypcje_files:
    if row['credibility'] >= 2: all_labels['t'].append(1)
    else: all_labels['t'].append(0)
     
    with codecs.open(fpath+row['id']+'.txt','r','utf8') as fr:
      all_texts['t'].append ( fr.read() )

print(len(all_texts['t']))

321
318


---
Veracity prediction using the Universal Sentence Encoder (USE) and SVM



In [ ]:
#!pip3 install tensorflow-gpu
!pip3 install --quiet tensorflow-hub
!pip3 install tensorflow_text #>=2.0.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.5 MB/s 
     |████████████████████████████████| 588.3 MB 19 kB/s 
     |████████████████████████████████| 1.7 MB 70.8 MB/s 
     |████████████████████████████████| 6.0 MB 61.0 MB/s 
     |████████████████████████████████| 439 kB 67.3 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.1

In [ ]:
import tensorflow_hub as hub
import numpy as np
import tensorflow_text

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")
#embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")
#embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/2")


Compute USE embeddings of each text:

In [ ]:

all_embedded_texts = {}
n=50

for text_type in ['p', 't']:

  all_embedded_texts[text_type] = []
  chunks=[all_texts[text_type][i:i + n] for i in range(0, len(all_texts[text_type]), n)]  
  for chunk in chunks:
    all_embedded_texts[text_type].extend( embed(chunk).numpy() )


In [ ]:
import numpy as np
from sklearn.model_selection import KFold, LeaveOneOut
from sklearn.metrics import classification_report
from sklearn.svm import NuSVC
from xgboost import XGBClassifier
from sklearn.dummy import DummyClassifier
import pickle
from scipy.stats import wilcoxon


#for kernel_type in ['rbf', 'linear']:
kernel_type = 'rbf'

use3_results = {}
for text_type in ['p', 't']:
  print(f"text_type: {text_type} ")
  embeddings = np.array(all_embedded_texts[text_type])
  labels = np.array(all_labels[text_type])

  use3_results[text_type] = []
  true_labels = []
  predicted_labels = []

  foldnr=0
  kf = KFold(n_splits=20, shuffle=True, random_state=42)
  #kf = LeaveOneOut()
  for train_index, test_index in kf.split(embeddings):

    X_train, X_test = embeddings[train_index], embeddings[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

    clf = NuSVC(nu=0.5, kernel=kernel_type)

    model = clf.fit(X_train, y_train.ravel())
    predictions = model.predict(X_test)

    use3_results[text_type].append( {'y_pred':predictions , 'y_true':y_test } )

    predicted_labels.extend(predictions)
    true_labels.extend(y_test)

    foldnr += 1

  print(classification_report(true_labels, predicted_labels, digits=2 ))

  dummy_clf = DummyClassifier(strategy="most_frequent")
  dummy_clf.fit(X_train, y_train.ravel())
  print(f'MF baseline: {dummy_clf.score(X_train, y_train.ravel())}')

  stat, p = wilcoxon(x=predicted_labels, y=true_labels)
  print('Statistics=%.3f, p=%.3f' % (stat, p))
  # interpret
  alpha = 0.05
  if p > alpha:
    print('Same distribution (fail to reject H0)')
  else:
    print('Different distribution (reject H0)')


text_type: p 
              precision    recall  f1-score   support

           0       0.58      0.53      0.55       153
           1       0.60      0.65      0.63       168

    accuracy                           0.60       321
   macro avg       0.59      0.59      0.59       321
weighted avg       0.59      0.60      0.59       321

MF baseline: 0.5245901639344263
Statistics=3799.000, p=0.219
Same distribution (fail to reject H0)
text_type: t 
              precision    recall  f1-score   support

           0       0.69      0.73      0.71       187
           1       0.58      0.53      0.56       131

    accuracy                           0.65       318
   macro avg       0.64      0.63      0.63       318
weighted avg       0.65      0.65      0.65       318

MF baseline: 0.5841584158415841
Statistics=2800.000, p=0.296
Same distribution (fail to reject H0)
